In [1]:

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

examples = [
    {
        "question": "영화 인셉션에 대해서 알려줘?",
        "answer": """
        | 항목         | 내용                                                                                                                    |
| ---------- | --------------------------------------------------------------------------------------------------------------------- |
| **감독**     | 크리스토퍼 놀란 (Christopher Nolan)                                                                                          |
| **주요 출연진** | 레오나르도 디카프리오, 조셉 고든 레빗, 톰 하디, 엘렌 페이지                                                                                   |
| **예산**     | 약 1억 6천만 달러                                                                                                           |
| **흥행 수익**  | 약 8억 2천 8백만 달러                                                                                                        |
| **장르**     | SF, 액션, 스릴러                                                                                                           |
| **시놉시스**   | 타인의 꿈속에 들어가 정보를 훔치는 기술이 가능해진 미래. 도미닉(디카프리오)은 이번엔 정보를 훔치는 게 아니라 생각을 심어야 한다. 현실과 꿈의 경계가 무너지는 가운데, 팀원들과 함께 미션을 수행하는 이야기. |
        """,
    },
    {
        "question": "영화 타이타닉에 대해서 알려줘",
        "answer": """
        | 항목         | 내용                                                                                               |
| ---------- | ------------------------------------------------------------------------------------------------ |
| **감독**     | 제임스 카메론 (James Cameron)                                                                          |
| **주요 출연진** | 레오나르도 디카프리오, 케이트 윈슬렛                                                                             |
| **예산**     | 약 2억 달러                                                                                          |
| **흥행 수익**  | 약 22억 달러                                                                                         |
| **장르**     | 로맨스, 드라마                                                                                         |
| **시놉시스**   | 1912년, ‘타이타닉’ 호에 탑승한 귀족 여성 로즈와 가난한 청년 잭이 서로 사랑에 빠지게 되지만, 배는 빙산에 충돌하면서 침몰하게 된다. 비극적인 사랑과 생존의 이야기. |

        """,
    },
    {
        "question": "영화 다크나이트에 대해서 알려줘",
        "answer": """
        | 항목         | 내용                                                                                            |
| ---------- | --------------------------------------------------------------------------------------------- |
| **감독**     | 크리스토퍼 놀란                                                                                      |
| **주요 출연진** | 크리스찬 베일, 히스 레저, 애런 에크하트, 마이클 케인                                                               |
| **예산**     | 약 1억 8천 5백만 달러                                                                                |
| **흥행 수익**  | 약 10억 달러                                                                                      |
| **장르**     | 액션, 범죄, 드라마                                                                                   |
| **시놉시스**   | 고담시를 지키는 배트맨과 혼돈의 화신 조커의 치열한 심리전. 악을 무너뜨리기 위해 자신조차 희생해야 하는 정의의 이면을 보여주는 영화. 히스 레저의 조커는 전설이 됨. |

        """,
    },
    {
        "question": "영화 기생충에 대해서 알려줘",
        "answer": """
        | 항목         | 내용                                                                                       |
| ---------- | ---------------------------------------------------------------------------------------- |
| **감독**     | 봉준호                                                                                      |
| **주요 출연진** | 송강호, 이선균, 조여정, 최우식, 박소담                                                                  |
| **예산**     | 약 1,100만 달러                                                                              |
| **흥행 수익**  | 약 2억 6천만 달러                                                                              |
| **장르**     | 드라마, 블랙 코미디, 스릴러                                                                         |
| **시놉시스**   | 가난한 가족이 부잣집에 하나씩 위장 취업하면서 벌어지는 이야기. 계층 간의 갈등과 사회적 불평등을 날카롭게 풍자하며, 아카데미 작품상 수상이라는 쾌거를 이룸. |

        """,
    },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,  
    example_prompt=example_prompt,
    suffix="Human: 영화 {movie}에 대해서 알려줘",
    input_variables=["movie"]
)
chain = few_shot_prompt | chat

chain.invoke({"movie": "어벤져스 엔드게임"}) 

AI:
        | 항목         | 내용                                                                                       |
| ---------- | ---------------------------------------------------------------------------------------- |
| **감독**     | 안소니 루소, 조 루소                                                                               |
| **주요 출연진** | 로버트 다우니 주니어, 크리스 에반스, 크리스 헴스워스, 스칼렛 요한슨, 마크 러팔로, 제레미 레너, 폴 러드, 조슈 브롤린 |
| **예산**     | 약 3억 5천만 달러                                                                              |
| **흥행 수익**  | 약 27억 8천만 달러                                                                             |
| **장르**     | 액션, SF, 모험                                                                                 |
| **시놉시스**   | 마블 시네마틱 유니버스의 최종장. 타노스에게 스냅을 당한 지 5년 후, 어벤져스는 다시 모여 타노스를 물리치기 위한 최후의 전투를 벌인다. 엔드게임. |

AIMessage(content='AI:\n        | 항목         | 내용                                                                                       |\n| ---------- | ---------------------------------------------------------------------------------------- |\n| **감독**     | 안소니 루소, 조 루소                                                                               |\n| **주요 출연진** | 로버트 다우니 주니어, 크리스 에반스, 크리스 헴스워스, 스칼렛 요한슨, 마크 러팔로, 제레미 레너, 폴 러드, 조슈 브롤린 |\n| **예산**     | 약 3억 5천만 달러                                                                              |\n| **흥행 수익**  | 약 27억 8천만 달러                                                                             |\n| **장르**     | 액션, SF, 모험                                                                                 |\n| **시놉시스**   | 마블 시네마틱 유니버스의 최종장. 타노스에게 스냅을 당한 지 5년 후, 어벤져스는 다시 모여 타노스를 물리치기 위한 최후의 전투를 벌인다. 엔드게임. |', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125', 'service_tier': 'default'}, i